This file will follow tutorial 
https://www.depends-on-the-definition.com/named-entity-recognition-conditional-random-fields-python/

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
def read_all_line( directory , file ):
    file = open( directory + "/" + file , "r" , encoding="utf-8" )
    contents = file.read()
    file.close()
    return contents

def idividual_read_file( raw_data , split_sentence , count ,remove = [] ):
    pre_data_frame = { "Sentence #" : [] , "Word" : [] , "Tag" : [] }
    for word in raw_data:
        if word in remove :
            None
        elif word in split_sentence :
            count += 1
        else: 
            check = word.find('(')
            if check > 0:
                pre_data_frame["Sentence #"].append( "Sentence: " + str(count) )
                pre_data_frame["Word"].append( word[ 0 : word.find('(')] )
                pre_data_frame["Tag"].append( word[ word.find('(') + 1 : word.find(')')] )
            else:
                pre_data_frame["Sentence #"].append( "Sentence: " + str(count) )
                pre_data_frame["Word"].append( word )
                pre_data_frame["Tag"].append( "O" )
    return pre_data_frame , count

def read_file( directory , list_file , count = 1 , individual = False ):
    data_frame = pd.DataFrame( { "Sentence #" : [] , "Word" : [] , "Tag" : [] } )
    if( individual ):
        word = read_all_line( directory , list_file ).split('|')
        pre_data_frame , count = idividual_read_file( word , # raw_data
                                              ["\n"] , # word show split sentence
                                              count , # order of sentence
                                              [ " " , '' , '\0'] ) # word to delete or prevent
        data_frame  = data_frame.append( pd.DataFrame( pre_data_frame ), ignore_index=True )
    else:
        for file in list_file :
            word = read_all_line( directory , file ).split('|')
            pre_data_frame , count = idividual_read_file( word ,
                                              ["\n"] ,
                                              count ,
                                              [ " " , '' , '\0'] )
            data_frame  = data_frame.append( pd.DataFrame( pre_data_frame ), ignore_index=True )
    return data_frame
def num_of_sentence( sentence ):
    words = sentence.split( " ")
    return int( words[1] )

In [ ]:
# Variable for collect name file and access file
directory = "corpus_directory"
train_files = ("Politic9.txt" , "Allcolumn.txt" , "98JUL5_1.txt" ,
                "98JUL5_2.txt" , "bkknews1.txt" , "Result1.txt" ,
                "Result2(corpus-1).txt" , "Result3(corpus-2).txt" )
test_files = ("result4.txt" , "Result_c_2_1.txt", "Result_c_2_2.txt" )

In [ ]:
data = read_file( directory , train_files , 1 )
data.fillna( method="ffill")
data.tail( 10 )

In [ ]:
def merge_all_tag( data ):
    for run in range(  0 , data.shape[0] ):
        if data.Tag[run] in ( '' ):
            data.Tag[run] = 'O'
        elif data.Tag[run] in ('loc_cont','loc_end','loc_start'):
            data.Tag[run] = 'loc'
        elif data.Tag[run] in ('org_cont','org_end','org_start'):
            data.Tag[run] = 'org'
        elif data.Tag[run] in ('per_cont','per_end','per_start'):
            data.Tag[run] = 'per'
        else:
            None

In [ ]:
merge_all_tag( data )
words = list( set(data["Word"].values ) )

In [ ]:
n_words = len( words )
n_sentence = num_of_sentence( data["Sentence #"][ data.shape[0]-1 ] )
print( "You have number of words is {} from {} sentences".format( n_words , n_sentence ))

In [ ]:
class SentenceGetter(object):
    
    def __init__(self, data):
        self.n_sent = 1
        self.data = data
        self.empty = False
        agg_func = lambda s: [(w, t) for w, t in zip( s["Word"].values.tolist(),
                                                      s["Tag"].values.tolist())]
        self.grouped = self.data.groupby("Sentence #").apply(agg_func)
        self.sentences = [s for s in self.grouped]
        
    def get_next(self):
        try:
            s = self.grouped[ "Sentence: {}".format( self.n_sent )]
            self.n_sent += 1
            return s  
        except:
            self.empty = True
            return None, None

In [ ]:
getter = SentenceGetter( data )

In [ ]:
sent= getter.get_next()

In [ ]:
print(sent)

ข้างบนนั้นจะดึงออกมาทีละประโยค โดยที่จะดึงคำออกมาพร้อมกับ tag นั้นเอง
ดังนั้นต่อมาเราจะดึงออกทั้งหมด ทุกประโยคเลย

In [ ]:
sentences = getter.sentences

In [ ]:
def word2features(sent, i):
    word = sent[i][0]

    features = {
        'bias': 1.0,
        'word.lower()': word.lower(),
        'word[-3:]': word[-3:],
        'word[-2:]': word[-2:],
        'word.isupper()': word.isupper(),
        'word.istitle()': word.istitle(),
        'word.isdigit()': word.isdigit(),
    }
    if i > 0:
        word1 = sent[i-1][0]
        postag1 = sent[i-1][1]
        features.update({
            '-1:word.lower()': word1.lower(),
            '-1:word.istitle()': word1.istitle(),
            '-1:word.isupper()': word1.isupper(),
        })
    else:
        features['BOS'] = True

    if i < len(sent)-1:
        word1 = sent[i+1][0]
        postag1 = sent[i+1][1]
        features.update({
            '+1:word.lower()': word1.lower(),
            '+1:word.istitle()': word1.istitle(),
            '+1:word.isupper()': word1.isupper(),
        })
    else:
        features['EOS'] = True

    return features


def sent2features(sent):
    return [word2features(sent, i) for i in range(len(sent))]

def sent2labels(sent):
    return [label for token, label in sent]

def sent2tokens(sent):
    return [token for token, label in sent]

ข้างบนเป็นการจัดรูปแบบ feature ของคำเฉยๆ ถ้าอยากรู้จะต้องอ่านเนื้อหาเกี่ยวกับ CRF เพิ่มเติมว่ามีแนวคิดอย่างไร คาดว่าจุดนี้อาจทำให้สามารถคำนวณโดยเอา Dict มาใช้ได้
https://medium.com/ml2vec/overview-of-conditional-random-fields-68a2a20fa541

In [ ]:
from sklearn_crfsuite import CRF

crf = CRF(algorithm='lbfgs',
          c1=0.1,
          c2=0.1,
          max_iterations=100,
          all_possible_transitions=False)

In [ ]:
# ref for first module https://scikit-learn.org/stable/modules/cross_validation.html
from sklearn.model_selection import cross_val_predict
from sklearn.metrics import classification_report
from sklearn_crfsuite.metrics import flat_classification_report

In [ ]:
X = [sent2features(s) for s in sentences]
y = [sent2labels(s) for s in sentences]

In [ ]:
pred = cross_val_predict(estimator=crf, X=X, y=y, cv=5)

In [ ]:
report = flat_classification_report(y_pred=pred, y_true=y)
print(report)

In [ ]:
crf.fit(X, y)

In [ ]:
import eli5

In [ ]:
eli5.show_weights(crf, top=30)